In [1]:
from main import *

CURRENT_PRICE = 96.94
CUREENT_DATE = datetime.now().strftime('%Y-%m-%d')
CURRENT_STOCK = 'MU'

print(SISTER_7_SLUGS)

results_dir = OutputPathSingleton.get_path()
api_client = StockOwnershipAPI(X_API_KEY, base_url='https://api.fintel.io/data/v/0.0/so/us/')
api_client.retrieve_sisters_data(slug_list=SISTER_7_SLUGS)
#api_client = StockOwnershipAPI(X_API_KEY, base_url='https://api.fintel.io/data/v/0.0/i/')
#api_client.retrieve_sisters_data(slug_list=['vanguard-group'])
sa = StockOwnershipAnalyzer(results_dir)
sa.load_data()
df_ori = sa.df
# get all symbols 
symbols = df_ori['symbol'].unique()

# print out the first 10 rows of df
df_ori.head(10)

# filter the data for TSLA
df = df_ori[df_ori['symbol'] == CURRENT_STOCK].copy()

# reset index
df.reset_index(drop=True, inplace=True)

# Convert the fileDate and effectiveDate columns to datetime
df['fileDate'] = pd.to_datetime(df['fileDate'])
df['effectiveDate'] = pd.to_datetime(df['effectiveDate'])

2025-03-29 11:49:24,810 - INFO - Skipping googl, file already exists.
2025-03-29 11:49:24,810 - INFO - Skipping amzn, file already exists.
2025-03-29 11:49:24,810 - INFO - Skipping meta, file already exists.
2025-03-29 11:49:24,811 - INFO - Skipping nvda, file already exists.
2025-03-29 11:49:24,811 - INFO - Skipping aapl, file already exists.
2025-03-29 11:49:24,811 - INFO - Skipping msft, file already exists.
2025-03-29 11:49:24,811 - INFO - Skipping tsla, file already exists.
2025-03-29 11:49:24,812 - INFO - Skipping intc, file already exists.
2025-03-29 11:49:24,812 - INFO - Skipping qcom, file already exists.
2025-03-29 11:49:24,812 - INFO - Skipping mu, file already exists.
2025-03-29 11:49:24,812 - INFO - Data retrieval completed.


['googl', 'amzn', 'meta', 'nvda', 'aapl', 'msft', 'tsla', 'intc', 'qcom', 'mu']


In [2]:
print(f'当前股票：{CURRENT_STOCK}')
print(f'当前日期：{CUREENT_DATE}')
print(f'当前价格：{CURRENT_PRICE}')

num_of_holding_funds = df['name'].unique()
print(f'持仓机构数量：{num_of_holding_funds.size:,.0f}')

当前股票：MU
当前日期：2025-03-25
当前价格：96.94
持仓机构数量：3,343


In [3]:

total_shares = df['shares'].sum()
total_effective_value = df['value'].sum()
total_today_value = total_shares * CURRENT_PRICE
total_cost = df['costBasis'].sum()
average_effective_price = total_effective_value / total_shares
average_purchase_price = total_cost * 1000 / total_shares
print(f'持仓总股数：{total_shares:,.0f}')
print(f'持仓总市值（公布日期）：{total_effective_value:,.2f}')
print(f'持仓平均价格（公布日期）：{average_effective_price:,.2f}')
print(f'持仓总市值（{CUREENT_DATE}）：{total_today_value:,.2f}')
print(f'持仓平均成本（期权本身价格是否计入不明）：{average_purchase_price:,.2f}')

持仓总股数：1,447,873,505
持仓总市值（公布日期）：120,469,618,566.00
持仓平均价格（公布日期）：83.20
持仓总市值（2025-03-25）：140,356,857,574.70
持仓平均成本（期权本身价格是否计入不明）：53.86


In [ ]:
# 亏本的股票
df_loss = df[df['sharePrice'] > CURRENT_PRICE]
df_loss_total_shares = df_loss['shares'].sum()
df_loss_total_value_today = df_loss_total_shares * CURRENT_PRICE
df_loss_total_cost = df_loss['costBasis'].sum() * 1000
print(f'亏本的机构数量：      {df_loss["name"].unique().size:,.0f}')
print(f'亏本的持仓数量：      {df_loss.shape[0]:,.0f}')
print(f'亏本的股票总市值：    {df_loss_total_value_today:,.2f}')
print(f'亏本的股票平均价格：  {(df_loss_total_cost / df_loss_total_shares):,.2f}')
print(f'亏本机构总成本：      {df_loss_total_cost:,.2f}')
print(f'亏损机构价值比率（%）：{df_loss_total_value_today / df_loss_total_cost * 100:,.2f}')

# 盈利的股票
df_profit = df[df['sharePrice'] <= CURRENT_PRICE]
df_profit_total_shares = df_profit['shares'].sum()
df_profit_total_value_today = df_profit_total_shares * CURRENT_PRICE
df_profit_total_cost = df_profit['costBasis'].sum() * 1000
print(f'盈利的机构数量：       {df_profit["name"].unique().size:,.0f}')
print(f'盈利的持仓数量：       {df_profit.shape[0]:,.0f}')
print(f'盈利的股票总市值：     {df_profit_total_value_today:,.2f}')
print(f'盈利的股票平均价格：   {(df_profit_total_cost / df_profit_total_shares):,.2f}')
print(f'盈利机构总成本：       {df_profit_total_cost:,.2f}')
print(f'盈利机构价值比率（%）：{df_profit_total_value_today / df_profit_total_cost * 100:,.2f}')

print(f'亏损总额 / 盈利总额（ 十亿 ）：{(df_loss_total_value_today - df_loss_total_cost) / 1000000000:,.2f} / {(df_profit_total_value_today - df_profit_total_cost) / 1000000000:,.2f}')

亏本的机构数量：      749
亏本的持仓数量：      793
亏本的股票总市值：    10,395,163,820.98
亏本的股票平均价格：  103.86
亏本机构总成本：      11,136,705,228.73
亏损机构价值比率（%）：93.34
盈利的机构数量：       2,029
盈利的持仓数量：       2,089
盈利的股票总市值：     129,759,435,229.52
盈利的股票平均价格：   52.42
盈利机构总成本：       70,162,440,410.59
盈利机构价值比率（%）：184.94
亏损总额 / 盈利总额（ 十亿 ）：-0.74 / 59.60


: 

,name,shares,prevShares
47,"Susquehanna International Group, Llp",54531700.0,45014900.0
299,"Susquehanna International Group, Llp",43597900.0,30286300.0
300,"Susquehanna International Group, Llp",1795518.0,3953176.0
